In [ ]:
!git clone https://github.com/referreira-wisc/digag2022.git

In [ ]:
import os
os.chdir('digag2022/Lab04')

#### Import required libraries

In [ ]:
from skimage.io import imread, imshow
import numpy as np
from scipy import stats

#### Open Image

In [ ]:
image = imread('image1.png')

#### Get image size

In [ ]:
#image = np.expand_dims(image, axis=-1)
#image = np.concatenate((image,image,image), axis=-1)
#print(image.shape)
image_size = image.shape
print(image_size)

#### Descriptive Statistics: Max, Min, Mean, Median, Mode - pixel values in each channel

In [ ]:
max_pixel = np.max(np.max(image, axis=0), axis=0)
min_pixel = np.min(np.min(image, axis=0), axis=0)
mean_pixel = np.mean(np.mean(image, axis=0), axis=0)
median_pixel = np.median(np.median(image, axis=0), axis=0)
mode_pixel = stats.mode(stats.mode(image)[0].squeeze())[0].squeeze()

print(max_pixel)
print(min_pixel)
print(mean_pixel)
print(median_pixel)
print(mode_pixel)

#### Visualize image

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
im = plt.imshow(image)

#### Image histogram for each RGB channel

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
image_histogram_red = plt.hist(image[:,:,0].flatten(), bins=256, range=[0, 256], facecolor='red')
plt.title('Red channel histogram')
plt.xlabel('Pixel value')
plt.ylabel('Number of pixels')
plt.subplot(2, 2, 2)
image_histogram_green = plt.hist(image[:,:,1].flatten(), bins=256, range=[0, 256], facecolor='green')
plt.title('Green channel histogram')
plt.xlabel('Pixel value')
plt.ylabel('Number of pixels')
plt.subplot(2, 2, 3)
image_histogram_blue = plt.hist(image[:,:,2].flatten(), bins=256, range=[0, 256], facecolor='blue')
plt.title('Blue channel histogram')
plt.xlabel('Pixel value')
plt.ylabel('Number of pixels')
plt.tight_layout()
plt.show()

#### Read each RGB channel

In [ ]:
red_channel = image[:,:,0]
green_channel = image[:,:,1]
blue_channel = image[:,:,2]

#### Define the upper and lower bounds of pixel values that corresponds to the cow, for each channel:

In [ ]:
red_lower = 120
red_upper = 255
green_lower = 130
green_upper = 255
blue_lower = 120
blue_upper = 255
#gray_upper = 250
#gray_lower = 255

#### Segmentation Part 1

In [ ]:
#using all channels
mask = (red_channel >= red_lower) & (red_channel <= red_upper) & \
       (green_channel >= green_lower) & (green_channel <= green_upper) & \
       (blue_channel >= blue_lower) & (blue_channel <= blue_upper)
# Set to 0 all pixels that are outside the defined bounds
rc = red_channel.copy()
rc[np.invert(mask)] = 0
gc = green_channel.copy()
gc[np.invert(mask)] = 0
bc = blue_channel.copy()
bc[np.invert(mask)] = 0

# Recombine separate color channels into a single, true color RGB image
segmented_image = np.stack((rc, gc, bc), axis=-1)

#### Segmentation Part 2 - Visualizing the original and segmented images

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.imshow(image)
plt.title('RGB - Original image')
plt.axis('off')
plt.subplot(2, 2, 2)
plt.imshow(segmented_image)
plt.title('RGB - Segmented image')
plt.axis('off')
plt.subplot(2, 2, 3)
plt.imshow(mask, cmap='gray')
plt.title('Binary mask')
plt.axis('off')
plt.tight_layout()
plt.show()

# #### Feature extraction - USE THIS PART FOR QUESTION 3!!

## open your image:

In [ ]:
from skimage.io import imread, imshow
import numpy as np
from scipy import stats

binaryImage = imread('g1bw.png')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
Segim = plt.imshow(binaryImage, cmap='gray')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go

from skimage import data, filters, measure, morphology
from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table
from skimage.transform import rotate

label_image = label(binaryImage)
props = regionprops_table(label_image, properties=('centroid',
                                                   'perimeter',
                                                       'area'))
pd.DataFrame(props)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go

from skimage import data, filters, measure, morphology
from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table
from skimage.transform import rotate

label_image = label(binaryImage)

mask1 = morphology.remove_small_objects(binaryImage, 7)
mask1 = morphology.remove_small_holes(binaryImage, 7)
labels = measure.label(mask1)

fig = px.imshow(binaryImage, binary_string=True)
fig.update_traces(hoverinfo='skip') # hover is only for label info


props = measure.regionprops(labels, mask1)
properties = ['area', 'eccentricity', 'perimeter', 'area']

# For each label, add a filled scatter trace for its contour,
# and display the properties of the label in the hover of this trace.
for index in range(1, labels.max()):
    label_i = props[index].label
    contour = measure.find_contours(labels == label_i, 0.5)[0]
    y, x = contour.T
    hoverinfo = ''
    for prop_name in properties:
        hoverinfo += f'<b>{prop_name}: {getattr(props[index], prop_name):.2f}</b><br>'
    fig.add_trace(go.Scatter(
        x=x, y=y, name=label_i,
        mode='lines', fill='toself', showlegend=False,
        hovertemplate=hoverinfo, hoveron='points+fills'))

plotly.io.show(fig)

